# BCI Comprehensive Data Alignment

# Section 1: Get epochs 

In [99]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from os import listdir
from mne.channels import make_standard_montage
from BCIAllFunction import BCIFuntions
from main_utilize import Unicorn

target_class = ["Left", "Right"] 
AllBCIClass = Unicorn(selectclass = target_class, desired_fz = 128, ch_pick = ['Fz','C3','Cz','C4','Pz'])

target_data_0 = "NutF8" #this subject will be test_set otherwise are train_set
calibrate_size = 10 # (trial)

condition_wLTL = "EA"
train_svm = True    

all_data = ["pipo","NutF8","AJpang","Aoomim","voen","pipo_HCI","Kawin"]

EEG_data = AllBCIClass.GetRawEDF(target_subjects= all_data, condition="Offline_Experiment") #Input data -> 250Hz with 6-32 Hz filtered + CAR

Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the

In [100]:
EEG_Epochs = AllBCIClass.GetEpoch(EEG_data ,tmin= -2.0, tmax= 6.0, crop = (0,4) ,baseline= (-0.5,0.0), band_pass=(6,32),trial_removal_th = 100)
calibrate_size = calibrate_size / EEG_Epochs[target_data_0]['Raw_Epoch'].shape[0]

Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
NutF8 -104.04128118288183 6
NutF8 86.93096401141673 6
NutF8 -104.31893713412965 11
NutF8 91.19854783269322 11
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
voen -1044.3428689131774 1
voen 694.2336071556814 1
voen -1039.3090824149497 2
voen 1021.4912607644604 2
voen -952.1519349924243 3
voen 1248.5144240059017 3
voen -1608.9284132711775 4
voen 965.6585709158061 4
voen -1275.969071501411 5
voen 920.2

c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: T

pipo_HCI -55.97212970445327 79
pipo_HCI 116.32214024347675 79
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']


c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()


# Section 2: Apply Data Alignment on RAW data

Note: After performing EA for all subjects, they share the same mean covariance matrix, i.e., the distances between the mean covariance matrices of different subjects are minimized (they become zero), and hence data distributions from different subjects become more similar.

In [101]:
alignmentMethod = "EA"

if alignmentMethod == "LA":
    AllBCIClass.ComputeLA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"
    count = 0

    for index in range(len(EEG_Epochs[target_data]['KMediod_label'])):
        if EEG_Epochs[target_data]['label'][index] == EEG_Epochs[target_data]['KMediod_label'][index]:
            count += 1
    print(count/len(EEG_Epochs[target_data]['KMediod_label']) * 100)
    
else:
    AllBCIClass.ComputeEA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"

In [102]:
target_data

'NutF8_test'

# Section 3 : CSP

In [103]:
CSP2D_Epoch = AllBCIClass.computeCSPFeatures(EEG_Epochs, target_subject = target_data)

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.


# Section 4 : T-SNE Plot

In [104]:
# AllBCIClass.TSNE_Plot(CSP2D_Epoch, target_subject= target_data)

# Frequency Selection (Don't finish)

In [105]:
# from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
# from sklearn.model_selection import train_test_split, GridSearchCV, KFold
# from mne.decoding import CSP
# from sklearn.svm import SVC
# from scipy import signal

# def butter_bandpass(lowcut,highcut,fs,order):
#     nyq = 0.5*fs
#     low = lowcut/nyq
#     high = highcut/nyq
#     b,a = signal.butter(order,[low,high],'bandpass')
#     return b,a

# def butter_bandpass_filter(data,lowcut = 6,highcut = 30, order = 4):
#     b,a = butter_bandpass(lowcut,highcut,128,order)
#     y = signal.filtfilt(b,a,data,axis=2)
#     return y

# def discriminative_frequency_band_selection(data, true_label, label_1, label_2, x_test, y_test):
#     Bh, Bl = 32, 6  # Initial upper and lower frequency limits
#     A1, A2 = 0, 0  # Initial accuracy values

#     # Step 1: Finding Bh
#     while A1 >= A2:
#         Bh -= 2
#         A2 = A1        
#         if(Bl >= Bh):
#             break
#         A1 = train_classifier(Bh, Bl, data, label_1, label_2, true_label, x_test, y_test)  # Design filter and train classifier using CSP

#     Bh += 1  # Adjust Bh back
#     A1 = train_classifier(Bh, Bl, data, label_1, label_2, true_label, x_test, y_test)

#     if A1 >= A2:
#         A2 = A1
#     else:
#         Bh = Bh + 1 
#         A1 = A2

#     # Step 2: Finding Bl
#     while A1 >= A2:
#         Bl += 2
#         A2 = A1
#         if(Bl >= Bh):
#             break
#         A1 = train_classifier(Bh, Bl, data, label_1, label_2, true_label, x_test, y_test)  # Design filter and train classifier using CSP

#     Bl -= 1  # Adjust Bl back
#     A1 = train_classifier(Bh, Bl, data, label_1, label_2, true_label, x_test, y_test)

#     if A1 >= A2:
#         A2 = A1
#     else:
#         Bh = Bh + 1 
#         A1 = A2

#     return Bh, Bl, A1


# def train_classifier(Bh, Bl, data, label_1, label_2, true_label, x_test, y_test):

#     init_data = data[np.where((true_label == label_1) | (true_label == label_2))]
#     init_label = true_label[np.where((true_label == label_1) | (true_label == label_2))]

#     print(init_data.shape, init_label.shape)

#     x_test = x_test[np.where((y_test == label_1) | (y_test == label_2))]
#     y_test = y_test[np.where((y_test == label_1) | (y_test == label_2))]

#     filtered_data = butter_bandpass_filter(init_data, lowcut= Bl, highcut= Bh)
#     x_test = butter_bandpass_filter(x_test, lowcut= Bl, highcut= Bh)

#     csp = CSP(n_components = 5, reg=None, log=None, rank= 'info')
#     csp.fit(filtered_data, init_label)

#     x_train = csp.transform(filtered_data)
#     x_test = csp.transform(x_test)

#     # Initialize SVM with a linear kernel
#     clf = SVC()

#     param_grid = {
#         'C':[1],
#         'kernel': ['rbf'],  # Example kernels
#     }

#     grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

#     grid_search.fit(x_train, init_label)
#     y_pred = grid_search.predict(x_test)

#     return accuracy_score(y_test, y_pred)


# def get_frequency_band(EEG_Epochs, target_data, condition = "noEA"):

#     train_data = None
#     train_label = None

#     tgt_data = target_data + "_test"

#     if condition == "noEA":
#         query = "Raw_Epoch"
#     else:
#         query = "EA_Epoch"

#     for sub in EEG_Epochs.keys():
#         if sub == target_data or sub == tgt_data:
#             pass

#         else:
#             if train_data is None:
#                 train_data = EEG_Epochs[sub][query]
#             else:
#                 train_data = np.concatenate((train_data, EEG_Epochs[sub][query]), axis=0)

#             if train_label is None:
#                 train_label = EEG_Epochs[sub]['label']
#             else:
#                 train_label = np.concatenate((train_label, EEG_Epochs[sub]['label']), axis=0)

#     indices = []
#     pairs = []
#     band_high = []
#     band_low = []
#     acc= []
#     class_name = target_class

#     if "Left" in class_name:
#         indices.append(0)
#     if "Right" in class_name:
#         indices.append(1)
#     if "Non" in class_name:
#         indices.append(2)
#     if "Feet" in class_name:
#         indices.append(3)    

#     train_data, x_test_temp, train_label, y_test_temp = train_test_split(train_data, train_label, test_size=0.3, random_state = 42, stratify=train_label)

#     # Nested loop to generate all pairs without reversing and without self-pairing
#     for i in range(len(indices)):
#         for j in range(i + 1, len(indices)):
#             pairs.append((indices[i], indices[j]))
#             Bh, Bl, A1 = discriminative_frequency_band_selection(data=train_data, true_label=train_label, label_1=indices[i], label_2=indices[j], x_test=x_test_temp, y_test=y_test_temp)
#             band_high.append(Bh)
#             band_low.append(Bl)
#             acc.append(A1)

#     csp_trained = []

#     for i in range(0,len(band_high)):
#         filter_x_train = butter_bandpass_filter(train_data ,lowcut= band_low[i], highcut=band_high[i])
#         csp = CSP(n_components = 5, reg=None, log=None, rank= 'info')
#         csp.fit(filter_x_train, train_label)
#         csp_trained.append(csp)

#     for sub in EEG_Epochs.keys():
#         stack_csp_train = []
#         for i in range(0,len(band_high)):
#             filtered = butter_bandpass_filter(EEG_Epochs[sub][query] ,lowcut= band_low[i], highcut=band_high[i])
#             filtered = csp_trained[i].transform(filtered)
#             stack_csp_train.append(filtered)
            
#         EEG_Epochs[sub]["Stack_epoch"] = np.hstack(np.array(stack_csp_train))

#     for i in range(len(indices)):
#         for j in range(i + 1, len(indices)):
#             print(f"The selected frequency band of {class_name[i]} vs {class_name[j]} is: Bl = {band_low[j-1+i]}, Bh = {band_high[j-1+i]}, Acc = {acc[j-1+i]}" )

#     for sub in EEG_Epochs.keys():
#         CSP2D_Epoch[sub]['Stack_epoch'] = EEG_Epochs[sub]['Stack_epoch']
#         EEG_Epochs[sub][query] = EEG_Epochs[sub]['Stack_epoch']

#     print(indices)

# get_frequency_band(EEG_Epochs, target_data_0, condition = "EA")

# Section 5: Model Training

# CSP+wLTL 

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Custom loss function
class CustomLossLL1(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        ws = self.get_weights_from_model()
        reg_term = self.regularization_term(ws)
        return ce_loss + self.lambda_t * reg_term

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        return model_weights[0]

    def regularization_term(self, ws):
        reg_term = tf.pow(tf.norm(ws, ord='euclidean'),2)
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL(X_train, y_train, lambd, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    _, sequential_indices = np.unique(y_train, return_inverse=True)
    y_one_hot = tf.keras.utils.to_categorical(sequential_indices, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')

        # Dense(5, input_shape=(X_train.shape[1],), activation='relu'),  # Hidden layer with 5 neurons
        # Dense(n_classes, activation='softmax')  # Output layer with 'n_classes' neurons
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL1(lambda_t, model)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if loss_value < lowest_loss:
            lowest_loss = loss_value
            best_weights = [layer.get_weights() for layer in model.layers]

        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    return best_weights[0], lowest_loss

def build_clf_params(data, target_subjects ,condition):

    for sub in data.keys():
        if (sub  != target_subjects) and  (sub != target_data_0): #Don't apply weight to target subject
            # Where the tranining data is stored
            if condition == "noEA":
                X = data[sub]['Raw_csp']
                y = data[sub]['Raw_csp_label']
                store_ws = 'ws_Raw'

            elif condition == "stack":
                X = data[sub]['Stack_epoch']
                y = data[sub]['Raw_csp_label']
                store_ws = 'ws_stack'

            else:
                X = data[sub]['EA_csp']
                y = data[sub]['EA_csp_label']
                store_ws = 'ws_EA'

            weights, loss = train_weight_LL(X_train=X, y_train=y, lambd= 0.1, num_tier=500, learning_rate= 0.005)
            print("weights of ", str(sub), ": ", weights)
            print("Lowest loss of ", str(sub), ": ", loss)
            data[sub][store_ws] = weights

build_clf_params(CSP2D_Epoch, target_subjects= target_data ,condition = "EA")

Epoch 0, Loss: 3.397817850112915
Epoch 20, Loss: 2.5311672687530518
Epoch 40, Loss: 2.086956739425659
Epoch 60, Loss: 1.8750338554382324
Epoch 80, Loss: 1.7859426736831665
Epoch 100, Loss: 1.7642723321914673
Epoch 120, Loss: 1.7483936548233032
Epoch 140, Loss: 1.7321845293045044
Epoch 160, Loss: 1.716222882270813
Epoch 180, Loss: 1.7009680271148682
Epoch 200, Loss: 1.68645441532135
Epoch 220, Loss: 1.6727676391601562
Epoch 240, Loss: 1.659987449645996
Epoch 260, Loss: 1.6481378078460693
Epoch 280, Loss: 1.6372194290161133
Epoch 300, Loss: 1.627213716506958
Epoch 320, Loss: 1.6180881261825562
Epoch 340, Loss: 1.6097997426986694
Epoch 360, Loss: 1.6023000478744507
Epoch 380, Loss: 1.5955376625061035
Epoch 400, Loss: 1.5894625186920166
Epoch 420, Loss: 1.5840245485305786
Epoch 440, Loss: 1.579177737236023
Epoch 460, Loss: 1.5748788118362427
Epoch 480, Loss: 1.5710891485214233
weights of  NutF8 :  [array([[-0.04969113,  0.41419408, -0.45505238, -0.1686008 ],
       [ 0.36918363, -0.4170037

NOTE:
Weights (array): These are the parameters that the model learns during training. They represent the strength of the connections between neurons. The shape of the weights array is (input_dim, output_dim).

Biases (array): These are additional parameters that are added to the outputs of each layer. The shape of the biases array is (output_dim,).

In [17]:
CSP2D_Epoch['pipo'].keys()

dict_keys(['Raw_csp', 'Raw_csp_label', 'EA_csp', 'EA_csp_label'])

In [21]:
# First define the kl divergence
def KL_div(P, Q):
    # First convert to np array
    P = np.array(P)
    Q = np.array(Q)

    # Then compute their means, datain shape of samples x feat
    mu_P = np.mean(P, axis=0)
    mu_Q = np.mean(Q, axis=0)    
    
    # Compute their covariance
    sigma_P = np.cov(P, rowvar=False)
    sigma_Q = np.cov(Q, rowvar=False)  

    print(mu_Q.shape)
    print(sigma_Q.shape)

    print(aaa)

    # Check the condition number
    if np.isnan(sigma_Q).any():
        pass
    else:
        condition_number = np.linalg.cond(sigma_Q)
        # If the condition number is too high, use a regularized version
        if condition_number > 1e10:  # Threshold to decide if it's ill-conditioned
            print("add small epsilon")
            epsilon = 1e-5  # Small regularization value
            sigma_Q += epsilon * np.eye(sigma_Q.shape[0])
            sigma_P += epsilon * np.eye(sigma_P.shape[0])

    diff = mu_Q - mu_P

    inv_sigma_Q = np.linalg.inv(sigma_Q)
    term1 = np.dot(np.dot(diff.T, inv_sigma_Q), diff)

    # Calculate the trace term trace(Sigma_Q^{-1} * Sigma_P)
    term2 = np.trace(np.dot(inv_sigma_Q, sigma_P))
    
    # Calculate the determinant term ln(det(Sigma_P) / det(Sigma_Q))
    det_sigma0 = np.linalg.det(sigma_P)
    det_sigma1 = np.linalg.det(sigma_Q)
    epsilon = 1e-10 
    term3 = np.log((det_sigma0+epsilon) / (det_sigma1+epsilon))
    
    # Dimensionality of the data
    K = mu_P.shape[0]
    
    # KL divergence
    kl_div = 0.5 * (term1 + term2 - term3 - K)
    
    return kl_div

In [22]:
# Compute kl divergence of target subject to each source subject
def compute_all_kl_div(data, target_subjects , condition):
    '''
    Parameter:
    data, is the whole data containing target and source data
    '''
    kl_div_score = []

    if condition == "noEA":
        target_data = 'Raw_csp'
        label_name = 'Raw_csp_label'

    elif condition == "stack":
        target_data = 'Stack_epoch'
        label_name = 'Raw_csp_label'

    else:
        target_data = 'EA_csp'
        label_name = 'EA_csp_label'
        
    # cal P from target data
    label_tgt =  data[target_subjects][label_name]
    P_left =  data[target_subjects][target_data][np.where(label_tgt == 0)]
    P_right = data[target_subjects][target_data][np.where(label_tgt == 1)]
    P_non = data[target_subjects][target_data][np.where(label_tgt == 2)]
    P_feet = data[target_subjects][target_data][np.where(label_tgt == 3)]

    tgt_data = target_subjects + "_test"

    #cal Q from each source subject
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            print(sub)
            label_src =  data[sub][label_name]
            Q_left =  data[sub][target_data][np.where(label_src == 0)]
            Q_right = data[sub][target_data][np.where(label_src == 1)]
            Q_non = data[sub][target_data][np.where(label_src == 2)]
            Q_feet = data[sub][target_data][np.where(label_src == 3)]

            kl_left = KL_div(P_left, Q_left)
            kl_right = KL_div(P_right, Q_right)
            kl_non = KL_div(P_non, Q_non)
            kl_feet = KL_div(P_feet, Q_feet)

            kl_div_temp = [kl_left, kl_right, kl_non, kl_feet]

            kl_div_score.append(kl_div_temp)

    data[target_subjects]['kl_div'] = kl_div_score


compute_all_kl_div(CSP2D_Epoch, target_subjects=target_data_0 ,condition = condition_wLTL) #target_sub for cal KL is calibrate set

NutF8
(5,)
(5, 5)


NameError: name 'aaa' is not defined

In [12]:
np.array(CSP2D_Epoch[target_data_0]['kl_div'])

array([[2.98303474, 2.33952316,        nan,        nan],
       [3.10634729, 2.92484184,        nan,        nan],
       [1.42967432, 1.68635534,        nan,        nan],
       [2.36528815, 2.33628648,        nan,        nan],
       [2.26677328, 1.66444477,        nan,        nan],
       [1.92099444, 1.80707149,        nan,        nan]])

In [13]:
def compute_similarity_weights(data, target_subjects):
    kl = data[target_subjects]['kl_div']
    KL_inv_left = []
    KL_inv_right = []
    KL_inv_non = []
    KL_inv_feet = []

    alpha_s = []
    eps = 0.0001
    
    #equation (9)
    for val in kl:
        if val != 0: 
            KL_inv_left.append(1/((val[0] + eps)**4))
            KL_inv_right.append(1/((val[1] + eps)**4))
            KL_inv_non.append(1/((val[2] + eps)**4))
            KL_inv_feet.append(1/((val[3] + eps)**4))

    print(KL_inv_left)
    print(KL_inv_right)
    
    for i in range(0,len(KL_inv_left)):
        temp = [KL_inv_left[i]/sum(KL_inv_left), KL_inv_right[i]/sum(KL_inv_right), KL_inv_non[i]/sum(KL_inv_non), KL_inv_feet[i]/sum(KL_inv_feet)]
        alpha_s.append(temp)

    alpha_s = np.array(alpha_s)
    print(np.array(alpha_s[:, ~np.isnan(alpha_s).any(axis=0)]))
                
    data[target_subjects]['alpha_s'] = alpha_s[:, ~np.isnan(alpha_s).any(axis=0)]

compute_similarity_weights(CSP2D_Epoch, target_subjects=target_data_0)

[0.012627242514735798, 0.010738510510669194, 0.2392930190268512, 0.03194407626297636, 0.03786962722957704, 0.0734184385916605]
[0.033374621820986716, 0.013662527845252596, 0.12362338222433357, 0.03355994660059298, 0.13026209946570444, 0.0937567469663231]
[[0.03110994 0.07793451]
 [0.02645664 0.03190395]
 [0.58955008 0.28867826]
 [0.07870114 0.07836727]
 [0.09330001 0.30418061]
 [0.18088219 0.2189354 ]]


In [14]:
def compute_ETL_and_mu_ws(data, target_subjects, condition):

    mu_ws = 0
    temp_ws = 0

    if condition == "noEA":
        ws_name = 'ws_Raw'
    elif condition == "stack":
        ws_name = 'ws_stack'
    else:
        ws_name = 'ws_EA'

    alpha_s = np.array(data[target_subjects]['alpha_s'])

    tgt_data = target_subjects + "_test"
    index_count = 0

    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0]
            mu_ws += ws * alpha_s[index_count]
            index_count += 1

    print(np.array(mu_ws))

    index_count = 0
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0]
            ws_min_mu = np.dot(((ws * alpha_s[index_count]) - mu_ws), np.transpose((ws * alpha_s[index_count]) - mu_ws))
            temp_ws += ws_min_mu #equation (11)
            index_count += 1

    print(np.array(temp_ws))
    
    den = temp_ws
    nom = np.trace(temp_ws) #Return the sum along diagonals of the array.
    Sigma_TL = den/nom

    data[target_subjects]['Sigma_TL'] = Sigma_TL
    data[target_subjects]['mu_ws'] = mu_ws

compute_ETL_and_mu_ws(CSP2D_Epoch, target_subjects = target_data_0, condition=condition_wLTL)

[[ 2.03243638e-01  1.95597816e-01]
 [ 4.87946131e-02 -3.51679631e-01]
 [ 4.96974916e-01  4.94377923e-04]
 [-3.03148699e-01  3.75366774e-01]
 [ 1.77596267e-01  4.98492852e-01]]
[[ 0.41581161 -0.27840064  0.46385884  0.01347705  0.58624869]
 [-0.27840064  0.57844223  0.02499062 -0.63020386 -0.72305472]
 [ 0.46385884  0.02499062  1.29000122 -0.66160188  0.39679994]
 [ 0.01347705 -0.63020386 -0.66160188  1.07009916  0.5738654 ]
 [ 0.58624869 -0.72305472  0.39679994  0.5738654   1.20141074]]


In [15]:
print(np.array(CSP2D_Epoch[target_data_0]['Sigma_TL']).shape)
print(np.array(CSP2D_Epoch[target_data_0]['mu_ws']).shape)

(5, 5)
(5, 2)


# Test ACC with target subject

In [16]:
# Custom loss function
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

class CustomLossLL2(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model, mu, sigma_TL):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model
        self.mu = tf.convert_to_tensor(mu, dtype=tf.float32)
        self.sigma_TL = tf.convert_to_tensor(sigma_TL, dtype=tf.float32)

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        wt = self.get_weights_from_model()
        reg_term = self.regularization_term(wt)

        return ce_loss + (self.lambda_t * tf.linalg.matmul(reg_term, wt))

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        return model_weights[0]

    def regularization_term(self, wt):
        diff = wt - self.mu
        reg_term = 0.5 * tf.linalg.matmul(tf.linalg.matmul(tf.linalg.inv(self.sigma_TL), diff), tf.transpose(diff))
        reg_term += 0.5 * tf.math.log(tf.linalg.det(self.sigma_TL))
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL2(X_train, y_train, lambd, mu, sigma_TL, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    _, sequential_indices = np.unique(y_train, return_inverse=True)
    y_one_hot = tf.keras.utils.to_categorical(sequential_indices, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')

        # Dense(5, input_shape=(X_train.shape[1],), activation='relu'),  # Hidden layer with 5 neurons
        # Dense(n_classes, activation='softmax')  # Output layer with 'n_classes' neurons 
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL2(lambda_t, model, mu, sigma_TL)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    best_model = None

    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        if (abs(loss_value) < lowest_loss):
            lowest_loss = abs(loss_value)
            best_model = model
            best_weights = [layer.get_weights() for layer in model.layers]

    return best_model, best_weights[0], lowest_loss

def GetConfusionMatrix(model, X_train, X_test, y_train, y_test):
    y_pred_prob = model.predict(X_train)
    y_pred = np.argmax(y_pred_prob, axis=1)
    _, y_train = np.unique(y_train, return_inverse=True)

    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    _, y_test = np.unique(y_test, return_inverse=True)

    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))

def tgt_test_wLTL(data, target_subjects ,condition):
    tgt_data = target_subjects + "_test"

    if condition == "noEA":
        X = data[target_subjects]['Raw_csp']
        y = data[target_subjects]['Raw_csp_label']
        X_test = data[tgt_data]['Raw_csp']
        y_test = data[tgt_data]['Raw_csp_label']
        store_ws = 'wt_Raw'

    elif condition == "stack":
        X = data[target_subjects]['Stack_epoch']
        y = data[target_subjects]['Raw_csp_label']
        X_test = data[tgt_data]['Stack_epoch']
        y_test = data[tgt_data]['Raw_csp_label']
        store_ws = 'wt_stack'

    else:
        X = data[target_subjects]['EA_csp']
        y = data[target_subjects]['EA_csp_label']
        X_test = data[tgt_data]['EA_csp']
        y_test = data[tgt_data]['EA_csp_label']
        store_ws = 'wt_EA'

    mu = data[target_subjects]['mu_ws']
    sigma_TL = data[target_subjects]['Sigma_TL']

    X_train = X
    y_train = y
    
    model, weights, loss = train_weight_LL2(X_train=X_train, y_train=y_train, mu =mu, sigma_TL=sigma_TL,  lambd= 0.1, num_tier=2000, learning_rate= 0.005)
    print("weights of ", str(target_subjects), ": ", weights)
    print("loss of ", str(target_subjects), ": ", loss)
    data[target_subjects][store_ws] = weights

    GetConfusionMatrix(model, X_train, X_test, y_train, y_test)

tgt_test_wLTL(CSP2D_Epoch, target_subjects= target_data_0 ,condition = condition_wLTL)

Epoch 0, Loss: -10.157519340515137
Epoch 20, Loss: -8.860455513000488
Epoch 40, Loss: -7.63074254989624
Epoch 60, Loss: -6.5803542137146
Epoch 80, Loss: -6.499639987945557
Epoch 100, Loss: -6.669436454772949
Epoch 120, Loss: -6.661418914794922
Epoch 140, Loss: -6.637301445007324
Epoch 160, Loss: -6.639811038970947
Epoch 180, Loss: -6.636754035949707
Epoch 200, Loss: -6.6317033767700195
Epoch 220, Loss: -6.627680778503418
Epoch 240, Loss: -6.623018741607666
Epoch 260, Loss: -6.618063449859619
Epoch 280, Loss: -6.612854957580566
Epoch 300, Loss: -6.607324123382568
Epoch 320, Loss: -6.601500034332275
Epoch 340, Loss: -6.595358848571777
Epoch 360, Loss: -6.588906288146973
Epoch 380, Loss: -6.582130432128906
Epoch 400, Loss: -6.5750322341918945
Epoch 420, Loss: -6.567597389221191
Epoch 440, Loss: -6.55981969833374
Epoch 460, Loss: -6.551698207855225
Epoch 480, Loss: -6.543229103088379
Epoch 500, Loss: -6.534393310546875
Epoch 520, Loss: -6.5251946449279785
Epoch 540, Loss: -6.51562881469726

# noEA + LDA

In [17]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0, condition = "noEA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.69      0.68      0.68       202
       Right       0.68      0.69      0.68       200

    accuracy                           0.68       402
   macro avg       0.68      0.68      0.68       402
weighted avg       0.68      0.68      0.68       402

Confusion matrix 
[[137  65]
 [ 63 137]]
Classification TEST DATA 
              precision    recall  f1-score   support

        Left       0.50      0.95      0.66        20
       Right       0.50      0.05      0.09        20

    accuracy                           0.50        40
   macro avg 

# noEA + SVM

In [18]:
if train_svm == True:
    AllBCIClass.classifyCSP_SVM(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0,condition = "noEA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Best parameters: {'C': 1, 'kernel': 'linear'}
Best cross-validation score: 0.664
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.67      0.68      0.67       202
       Right       0.67      0.66      0.66       200

    accuracy                           0.67       402
   macro avg       0.67      0.67      0.67       402
weighted avg       0.67      0.67      0.67       402

Confusion matrix 
[[137  65]
 [ 69 131]]
Classification TEST DATA 
              precision    recall  f1-score   support

        Left       0.50      0.95      0.66        20
       Right       0.50      0.05      0.

# EA+LDA

In [19]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0, condition = "EA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.73      0.73      0.73       202
       Right       0.73      0.72      0.73       200

    accuracy                           0.73       402
   macro avg       0.73      0.73      0.73       402
weighted avg       0.73      0.73      0.73       402

Confusion matrix 
[[148  54]
 [ 55 145]]
Classification TEST DATA 
              precision    recall  f1-score   support

        Left       0.73      0.55      0.63        20
       Right       0.64      0.80      0.71        20

    accuracy                           0.68        40
   macro avg 

In [20]:
if train_svm == True:
    AllBCIClass.classifyCSP_SVM(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0,condition = "EA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Best parameters: {'C': 1, 'kernel': 'rbf'}
Best cross-validation score: 0.721
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.78      0.77      0.77       202
       Right       0.77      0.78      0.77       200

    accuracy                           0.77       402
   macro avg       0.77      0.77      0.77       402
weighted avg       0.77      0.77      0.77       402

Confusion matrix 
[[155  47]
 [ 45 155]]
Classification TEST DATA 
              precision    recall  f1-score   support

        Left       0.62      0.40      0.48        20
       Right       0.56      0.75      0.64 